# Initialize cluster and Moving randomly any molecule
# avoiding that any atoms overlaps each other:

In [1]:
# crating the path (PYTHONPATH) to our module.
# assuming that our 'amcess' directory is out ('..') of our current directory 
import os
import sys
module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
def visualize(
        xyz_file: str, 
        sphere_radius: int, 
        sphere_center: tuple,
        style='sphere',
        ):
        """
        Visualize the molecule with a sphere around the molecule.
        """

        sr = sphere_radius
        cx, cy, cz = sphere_center

        if not xyz_file:
                return None

        # starting visualization
        xyz_view = py3Dmol.view() # width=400, height=300)#, linked=False, viewergrid=(2,2))
        xyz_view.addModelsAsFrames(xyz_file,'xyz')
        # xyz_view.setStyle({style: {'radius': 0.6}})

        if style == 'stick':
                xyz_view.setStyle({'stick': {'radius': 0.15}})
        else:
                xyz_view.setStyle({'sphere': {'radius': 0.8}})


        xyz_view.zoomTo()

        # animation base on several XYZ coordinates snapshots
        xyz_view.animate({'loop': "forward", 'speed': 1, 'reps': 1})

        # sphere center
        sc = {"x": cx, "y": cy, "z": cz}

        # 20% to check visually that everything is inside
        xyz_view.addSphere({'center': sc, 
                'radius': sr, 
                'color' :'yellow',
                'alpha': 0.5,
                })

        # cartesian 3D axes
        x_axis = {'start': {'x': cx-sr, 'y':cy, 'z':cz}, 'end': {'x': cx+sr, 'y':cy, 'z':cz}}
        y_axis = {'start': {'x': cx, 'y':cy-sr, 'z':cz}, 'end': {'x': cx, 'y':cy+sr, 'z':cz}}
        z_axis = {'start': {'x': cx, 'y':cy, 'z':cz-sr}, 'end': {'x': cx, 'y':cy, 'z':cz+sr}}

        xyz_view.addLine(x_axis)
        xyz_view.addLine(y_axis)
        xyz_view.addLine(z_axis)

        xyz_view.addLabel("x", {
                'position':x_axis["end"],
                'inFront':'true',
                'fontSize':20,
                'showBackground':'false',
                'fontColor': 'black',
                })
        xyz_view.addLabel("y", {
                'position':y_axis["end"],
                'inFront':'true',
                'fontSize':20,
                'showBackground':'false',
                'fontColor': 'black',
                })
        xyz_view.addLabel("z", {
                'position':z_axis["end"],
                'inFront':'true',
                'fontSize':20,
                'showBackground':'false',
                'fontColor': 'black',
                })

        return xyz_view

# xyz_view.show()

In [3]:
# py3Dmol: a simple IPython/Jupyter widget to embed an interactive 
# 3Dmol.js viewer in a notebook.
import py3Dmol

# importing de Molecule Class
from amcess.base_molecule import Molecule, Cluster

In [4]:
hf = {"atoms": [("H", 0, 0, 0), ("F", 1, 0, 0)]}
cc = {"atoms": [("C", 0, 0, 0), ("C", 1.2, 0, 0)]}
ab = Cluster(hf, cc)

In [5]:
xyz_data = ab.xyz.replace('\t', '').split('\n')
xyz_data = xyz_data[2:]
xyz_data = ';'.join(xyz_data)

print(xyz_data)

H          0.00000000     0.00000000     0.00000000;F          1.00000000     0.00000000     0.00000000;C          0.00000000     0.00000000     0.00000000;C          1.20000000     0.00000000     0.00000000;


In [6]:
ab_xyz = ""

ab.sphere_radius = 4
# ab.freeze_molecule = 0

for _ in range(100):
    mol = ab.random_generator.choice(ab.total_molecules)

    ab = ab.move_molecule(
        molecule=mol,
        max_step=3,
        max_rotation=10,
        max_closeness=0.5,
    )
    # print(ab.xyz)
    ab_xyz += ab.xyz

# file_name = "cluster_test.xyz"
# with open(file_name, "w") as f:
#     f.write(ab_xyz)

In [7]:
view = visualize(        
    xyz_file = ab_xyz,
    sphere_radius = ab.sphere_radius * 1.2,
    sphere_center = ab.sphere_center,
)
view.show()


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [8]:
ab_xyz = ""
ab.sphere_radius = 4
ab.freeze_molecule = None

In [9]:
ab.freeze_molecule = 0

for _ in range(100):
    mol = ab.random_generator.choice(ab.total_molecules)

    ab = ab.move_molecule(
        molecule=mol,
        max_step=4.0,
        max_rotation=30,
        max_closeness=1.0,
    )
    # print(ab.xyz)
    ab_xyz += ab.xyz

# file_name = "cluster_test.xyz"
# with open(file_name, "w") as f:
#     f.write(ab_xyz)

# print(ab.freeze_molecule)
# print(ab.cluster_dictionary)

view = visualize(        
    xyz_file = ab_xyz,
    sphere_radius = ab.sphere_radius * 1.2,
    sphere_center = ab.sphere_center,
)
view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [10]:
import numpy as np

def overlapping(
    first_coordinates: list,
    second_coordinates: list,
    max_closeness: float = 1.0,
) -> bool:
    for first_atom in first_coordinates:
        for second_atom in second_coordinates:
            distance = np.linalg.norm(
                np.asarray(first_atom) - np.asarray(second_atom)
            )

            if distance < max_closeness:
                return True

    return False

In [11]:

def initialize_cluster(mol, max_closeness: float = 1.0) -> object:
    # center of mass coordinates
    sc_x = mol.sphere_center[0]
    sc_y = mol.sphere_center[1]
    sc_z = mol.sphere_center[2]

    # initializing a new cluster moving the first molecule
    # to the center of the cluster sphere
    molecule = mol.get_molecule(0)
    new_cluster = molecule.translate(0, sc_x, sc_y, sc_z)

    for i in range(1, mol.total_molecules):
        # moving the next single molecule into the cluster sphere
        molecule = mol.get_molecule(i).translate(0, sc_x, sc_y, sc_z)

        if overlapping(molecule.coordinates, new_cluster.coordinates):
            new_cluster += molecule
            new_cluster = new_cluster.move_molecule(
                i,
                max_step=None,
                max_rotation=None,
                max_closeness=max_closeness,
            )
        else:
            new_cluster += molecule

    return Cluster(
        new_cluster,
        freeze_molecule=mol.freeze_molecule,
        sphere_radius=mol.sphere_radius,
        sphere_center=mol.sphere_center,
    )

In [12]:
hf6 = 6 * Cluster(hf)

hf6.sphere_radius = 4
hf6.sphere_center = 10, 10, 10

print(hf6.seed)

369072


In [13]:
hf6.seed = 932570

new_hf6 = initialize_cluster(hf6, max_closeness=4.0)
print(new_hf6.xyz)

	12
-- charge=0 and multiplicity=1 --
H     	    10.00000000	    10.00000000	    10.00000000
F     	    11.00000000	    10.00000000	    10.00000000
H     	     9.13276563	     9.50940864	    13.91396910
F     	     9.91031068	     8.88441486	    13.84464035
H     	    10.43015547	    14.05650701	    10.38777975
F     	    10.43429075	    13.74553991	    11.33819143
H     	     8.64674856	     6.23922512	     9.74787905
F     	     8.45073655	     6.50465424	     8.80388399
H     	     6.02236477	    10.64264606	    10.75086616
F     	     6.16231858	    11.12577079	     9.88657256
H     	     9.38895027	    11.28186978	     6.14227187
F     	     9.77071366	    10.36533990	     6.02298362



In [14]:
new_view = visualize(
    xyz_file = new_hf6.xyz,
    sphere_radius = new_hf6.sphere_radius * 1.2,
    sphere_center = new_hf6.sphere_center,
)
new_view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [15]:
water_molecule=[
    ("O", 0, 0, 0), ("H", 0.58708, 0.75754, 0), ("H", -0.58708, 0.75754, 0)
    ]

w2 = Cluster(water_molecule, water_molecule)

# the water dimer was initialized with the same water molecule, so
# both molecules has the same coordinates an those atoms overlap each other
print(w2.xyz)

	6
-- charge=0 and multiplicity=1 --
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000



In [16]:
#let's move any molecule randomly to avoid averlaping
new_w2 = w2.move_molecule(
    molecule = 1,
    max_step = 1.4,
    max_rotation = 30,
    max_closeness = 1.2,
)
print(new_w2.xyz)

	6
-- charge=0 and multiplicity=1 --
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     1.03661403	    -1.34103709	     1.20970284
H     	     1.18995912	    -0.46393862	     1.56426361
H     	     0.17508697	    -1.05234219	     1.51458996



In [17]:
new_w2 = w2.move_molecule(1)
print(new_w2.xyz)

	6
-- charge=0 and multiplicity=1 --
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     0.82297356	    -1.04948405	     0.94257402
H     	     0.97631865	    -0.17238558	     1.29713479
H     	    -0.03855350	    -0.76078915	     1.24746114



In [18]:
from copy import deepcopy
from numpy import random
random_gen = random.default_rng(1234)

# freeze ibuprofen molecule to avoid be moved (just water molecules dancing)
# w2.freeze_molecule = 0

w2.sphere_radius = 4

new_w2 = deepcopy(w2)
new_w2 = new_w2.initialize_cluster(max_closeness=1)

# to save snapshot and show as a movie
w2_xyz = ""

total_steps = 100

for i in range(total_steps):
    # # molecule to be selected between [0, w.total_molecules]
    # # freeze molecules will not be moved or rotated
    mol = random_gen.choice(new_w2.total_molecules)

    print(new_w2.xyz)


    # saving coordinates as a string
    w2_xyz += new_w2.xyz
    # if i % 10 == 0: 
    #     w2_xyz += new_w2.xyz

    #----------------------------------------------------------

    new_w2 = new_w2.move_molecule(
        molecule=mol,
        max_step=2,
        max_rotation=30,
        max_closeness=0.5,
    )

# # file_name = "w2.xyz"
# # with open(file_name, "w") as f:
# #     f.write(w2_xyz)

	6
-- charge=0 and multiplicity=1 --
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     1.03226563	     0.42112861	    -0.99281594
H     	     1.32357490	     1.32295388	    -0.85005784
H     	     0.34909078	     0.95964071	    -1.39506788

	6
-- charge=0 and multiplicity=1 --
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	    -0.11789918	    -0.47417899	    -0.95259411
H     	    -0.30900602	     0.42453611	    -0.67997874
H     	    -0.80319116	    -0.18197588	    -1.55552198

	6
-- charge=0 and multiplicity=1 --
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     0.13010315	     1.62303708	    -0.97619329
H     	

In [19]:
new_view = visualize(
    xyz_file = w2_xyz,
    sphere_radius =w2.sphere_radius * 1.2,
    sphere_center = w2.sphere_center,
)
new_view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [20]:
# for several molecules we could move randomly one-by-one
# or we could just `initialize` this cluster with all its molecules 
# moved around avoiding overlapping
w6 = 3 * w2

# let's define the spherical boundary conditions
w6.sphere_radius = 10

# maximum closeness is the minimum distance for two atoms
new_w6 = w6.initialize_cluster(max_closeness=3)

print(new_w6.xyz)

	18
-- charge=0 and multiplicity=1 --
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	    -3.28043902	     1.93314492	    -3.47768415
H     	    -2.73756909	     2.71968747	    -3.40576164
H     	    -3.73473352	     2.58689986	    -4.01130460
O     	    -3.22715854	    -2.78386540	     2.00316037
H     	    -2.36103202	    -2.37885726	     1.93741055
H     	    -3.40405787	    -1.84205070	     1.98827812
O     	     1.40789105	     0.06570801	    -3.10668424
H     	     1.87724107	     0.86846927	    -3.33866417
H     	     0.72162054	     0.66878420	    -3.39627304
O     	    -3.60265157	     1.92387606	     1.60238459
H     	    -2.77120480	     2.40023476	     1.58482226
H     	    -3.85043279	     2.84192806	     1.72202890
O     	     3.03589885	    -2.24030464	     1.87810180
H     	     3.24377707	    -1.31328509	     2.00439977
H     	     2.21136620	    

!pip install py3Dmol
import py3Dmol

In [21]:
new_view = visualize(
    xyz_file = new_w6.xyz,
    sphere_radius =new_w6.sphere_radius * 1.2,
    sphere_center = new_w6.sphere_center,
)
new_view.show()


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [22]:
from data.molecules_coordinates import water, ibuprofen 

# let's create a molecular object for the ibuprofen
ibu = Molecule(ibuprofen["atoms"])

# let's create a molecular object for one water molecule
w = Molecule(water["atoms"])

In [23]:
w6_overlap = Cluster(6 * w)

# let's initialize a cluster of six water without overlapping
w6 = Cluster(6 * w).initialize_cluster(max_closeness=2)


print(w6.xyz)


	18
-- charge=0 and multiplicity=1 --
O     	     0.00000000	     0.00000000	     0.00000000
H     	     0.58708000	     0.75754000	     0.00000000
H     	    -0.58708000	     0.75754000	     0.00000000
O     	     2.05659152	    -2.19081339	     1.25471535
H     	     2.92094947	    -1.78665556	     1.16487346
H     	     1.87240397	    -1.26045525	     1.11673544
O     	     2.09007964	     1.89752660	     1.30135733
H     	     2.68924517	     2.60146511	     1.04837306
H     	     1.52785756	     2.66802846	     1.20774642
O     	    -1.77605665	     0.35724057	    -1.71101394
H     	    -0.98520996	     0.78642462	    -2.04100232
H     	    -1.95273988	     1.29387243	    -1.61084621
O     	    -0.73327843	    -1.52004979	     1.86883959
H     	    -0.54035974	    -0.61846170	     2.13047117
H     	    -1.51905680	    -1.02750980	     1.62699978
O     	    -2.07329845	     2.07623959	     1.55365597
H     	    -1.62123932	     2.86280287	     1.86267138
H     	    -2.73711198	    

In [24]:
new_view = visualize(
    xyz_file = w6.xyz,
    sphere_radius = 5,
    sphere_center = w6.sphere_center,
)
new_view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [25]:
ibu_w6 = Cluster(ibu + w6)

# let's define a sphere with center around the carboxylic acid group
ibu_w6.sphere_center = -3, 0.2, 0.2
ibu_w6.sphere_radius = 4

ibu_w6 = ibu_w6.initialize_cluster(max_closeness=2)

print(ibu_w6.xyz)

	50
-- charge=0 and multiplicity=1 --
C     	    -1.56098300	    -1.10947200	     1.17219000
C     	    -2.94477900	    -1.24360900	     1.33412100
C     	    -3.84450500	    -0.79651100	     0.35585200
C     	    -3.30141000	    -0.20160700	    -0.79830200
C     	    -1.92211300	    -0.07098900	    -0.95773900
C     	    -1.02102000	    -0.52007300	     0.02281900
H     	    -0.89290600	    -1.46731500	     1.95548700
H     	    -3.33425300	    -1.70043400	     2.24251900
H     	    -3.98572600	     0.18365200	    -1.54865500
H     	    -1.53569500	     0.39530700	    -1.86436000
C     	    -5.34102100	    -0.97417500	     0.53451400
H     	    -5.51739900	    -1.32551300	     1.55744700
C     	     0.47407100	    -0.34878900	    -0.15756300
H     	     0.76256300	    -0.69849900	    -1.16041200
H     	     1.00698600	    -0.99284600	     0.55851800
C     	    -5.90009800	    -2.02131200	    -0.45068600
H     	    -5.37923800	    -2.98385700	    -0.34592400
H     	    -6.96972400	    

In [26]:
new_view = visualize(
    xyz_file = ibu_w6.xyz,
    sphere_radius = ibu_w6.sphere_radius * 1.2,
    sphere_center = ibu_w6.sphere_center,
    style='stick',
)
new_view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [27]:
from numpy import random
random_gen = random.default_rng(1234)

new_ibu_w6 = deepcopy(ibu_w6)

# to save snapshot and show as a movie
new_ibu_w6_xyz = ""

# freeze ibuprofen molecule to avoid be moved (just water molecules dancing)
# new_ibu_w6.freeze_molecule = 0

new_ibu_w6.sphere_radius = 3 
new_ibu_w6.sphere_center = -6.12410200, 0.37976300, 0.38520700


# to avoid overlapping and any molecule out of the sphere
ibu_w6 = ibu_w6.initialize_cluster(max_closeness=2)

total_steps = 100

for i in range(total_steps):
    # # molecule to be selected between [0, w.total_molecules]
    # # freeze molecules will not be moved or rotated
    mol = random_gen.choice(new_ibu_w6.total_molecules)

    if mol == 0:
        continue

    # saving coordinates as a string
    new_ibu_w6_xyz += new_ibu_w6.xyz
    # if i % 10 == 0: 
        # new_ibu_w6_xyz += new_ibu_w6.xyz

    # #----------------------------------------------------------

    new_ibu_w6 = new_ibu_w6.move_molecule(
        molecule=mol,
        max_closeness=0.5,
        max_step=2,
        max_rotation=30,    
    )

    #----------------------------------------------------------


    # printing current step
    print(
        f"\r progress {100.0*((i + 1)/total_steps):.2f}"
        f" % -- step {i + 1}/{total_steps}", end=''
        )
# -------------------------------------------------------
print("\n *** JOB DONE ***")
print(f"after {total_steps} steps\n")
# print(new_ibu_w6_xyz)

# open("new_ibu_w6.xyz", "w").write(new_ibu_w6_xyz)

 progress 100.00 % -- step 100/100
 *** JOB DONE ***
after 100 steps



In [28]:
new_view = visualize(
    xyz_file = new_ibu_w6_xyz,
    sphere_radius = new_ibu_w6.sphere_radius * 1.2,
    sphere_center = new_ibu_w6.sphere_center,
    style='stick',
)
new_view.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol